In [1]:
import sys, os

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from src.io.input import loadMidiFile
from src.io.record_midi import MidiInput
from src.io.output import pianoRoll, playPrettyMidi
from src.io.conversion import note_seq_to_pretty_midi
from src import analysis
from src.adaptation import Adaptation
from src.datatypes.melody_data import MelodyData
from definitions import SequenceType
from src.generation.generators import MusicVAEGenerator

pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# Initialize the generation model
(takes up to 3 minutes)

In [2]:
vae = MusicVAEGenerator()

[GEN] Initializing Music VAE with checkpoint 'cat-mel_2bar_big'...
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

Instructions for u

# Preparation of the Adaptation Pipeline
## Instantiate Adaptation Object


In [3]:
a = Adaptation()

## Print available adaptation operations

In [4]:
for op in a.available_operations:
    print(op.__name__, ":", op.__doc__)

StartAndEndOnCTOperation : Creates a chord based on the estimated key of the control sequence and transposes the first and last note of the base sequence so they are chord tones (e.g. 'c', 'e' or 'g' for a C major chord).
TransposeNotesOperation : Estimates the key of both sequences and transposes every single note of the base sequence separately to the closest pitch that fits the key of the control sequence.
TransposeSequenceOperation : Estimates the key of both sequences and transposes the base sequence to be in the same key as the control sequence.


## Construct a pipeline

In [5]:
a.construct_pipeline(['StartAndEndOnCTOperation', 'TransposeSequenceOperation'])

# Load input

In [6]:
input_seq = loadMidiFile('../midi/examples/monophonic/twinkle1_4b.mid')

C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pretty_midi\pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


# Generate base sequence for adaptation
+ parameters: number of beats and temperature

In [7]:
gen_base = vae.generate(16, 0.2)

# Create MelodyData objects and start adaptation

In [8]:
mel_input = MelodyData(input_seq, SequenceType.REC_INPUT, {})
mel_gen_base = MelodyData(note_seq_to_pretty_midi(gen_base['sequence']), SequenceType.GEN_BASE, { 'generation': gen_base['meta'] })
pianoRoll(mel_gen_base.sequence)

result, control = a.adapt(mel_gen_base, mel_input)

Loading BokehJS ...

# Print & play the results

In [9]:
pianoRoll(mel_gen_base.sequence)
pianoRoll(result.sequence)
print(result.meta)

Loading BokehJS ...

Loading BokehJS ...

{'generation': {'gen_dur': 3.7506520748138428, 'model': 'MusicVAE', 'checkpoint': 'cat-mel_2bar_big', 'temperature': 0.2}, 'adaptation': {'steps': [{'name': 'StartAndEndOnCTOperation', 'duration': 0.003999233245849609, 'intermediate_result': <music21.stream.Score 0x1ed3c034cc8>}, {'name': 'TransposeSequenceOperation', 'duration': 0.0398252010345459, 'intermediate_result': <music21.stream.Score 0x1ed3c137708>}], 'total_duration': 0.04382443428039551}}


In [10]:
playPrettyMidi(mel_gen_base.sequence)

In [11]:
playPrettyMidi(result.sequence)

In [12]:
control.analysis

{'key': <music21.key.Key of C major>}

In [13]:
result.analysis

{'pitch_span': (<music21.pitch.Pitch G4>, <music21.pitch.Pitch C#5>),
 'key': <music21.key.Key of F# major>}

In [14]:
result.sequence.instruments[0].notes

[Note(start=0.000000, end=0.375000, pitch=61, velocity=80),
 Note(start=0.750000, end=1.000000, pitch=62, velocity=80),
 Note(start=1.000000, end=1.125000, pitch=62, velocity=80),
 Note(start=1.250000, end=1.500000, pitch=65, velocity=80),
 Note(start=1.750000, end=2.000000, pitch=65, velocity=80),
 Note(start=2.000000, end=2.125000, pitch=65, velocity=80),
 Note(start=2.250000, end=2.750000, pitch=65, velocity=80),
 Note(start=2.750000, end=8.000000, pitch=67, velocity=80)]

In [15]:
result.meta

{'generation': {'gen_dur': 3.7506520748138428,
  'model': 'MusicVAE',
  'checkpoint': 'cat-mel_2bar_big',
  'temperature': 0.2},
 'adaptation': {'steps': [{'name': 'StartAndEndOnCTOperation',
    'duration': 0.003999233245849609,
    'intermediate_result': <music21.stream.Score 0x1ed3c034cc8>},
   {'name': 'TransposeSequenceOperation',
    'duration': 0.0398252010345459,
    'intermediate_result': <music21.stream.Score 0x1ed3c137708>}],
  'total_duration': 0.04382443428039551}}

In [16]:
result.meta['adaptation']['steps'][0]['intermediate_result'].show('text')

{0.0} <music21.stream.Part 0x1ed3c02d488>
    {0.0} <music21.tempo.MetronomeMark animato Quarter=120.0>
    {0.0} <music21.note.Note G>
    {1.5} <music21.note.Note G#>
    {2.0} <music21.note.Note G#>
    {2.5} <music21.note.Note B>
    {3.5} <music21.note.Note B>
    {4.0} <music21.note.Note B>
    {4.5} <music21.note.Note B>
    {5.5} <music21.note.Note C#>
